In [1]:
from sympy.stats import DiscreteMarkovChain, TransitionMatrixOf
from sympy import Matrix, MatrixSymbol, Eq
from sympy.stats import P

In [43]:
# Markov Matrix
M = Matrix([
    [0.15, 0.50, 0.25, 0.80, 0.00, 0.25],  # So1
    [0.05, 0.25, 0.00, 0.00, 0.00, 0.00],  # Sd1
    [0.00, 0.10, 0.50, 0.00, 0.00, 0.00],  # Su1
    [0.80, 0.00, 0.00, 0.15, 0.50, 0.25],  # So2
    [0.00, 0.15, 0.00, 0.05, 0.25, 0.00],  # Sd2
    [0.00, 0.00, 0.25, 0.00, 0.25, 0.50]   # Su2
])

# make sure the cols sum up to 1
for i in range(M.shape[1]):
    assert sum(M.col(i)) == 1.0, 'Assertion failed for column {}, sum of probabilities is {}'.format(i, sum(M.col(i)))

# Transpose the damn thing, because Sympy is row-dominant not col-dominant
M = M.T
    
# symbol matrix representation of M
MS = MatrixSymbol('M', M.shape[0], M.shape[1])
Matrix(MS)

    

Matrix([
[M[0, 0], M[0, 1], M[0, 2], M[0, 3], M[0, 4], M[0, 5]],
[M[1, 0], M[1, 1], M[1, 2], M[1, 3], M[1, 4], M[1, 5]],
[M[2, 0], M[2, 1], M[2, 2], M[2, 3], M[2, 4], M[2, 5]],
[M[3, 0], M[3, 1], M[3, 2], M[3, 3], M[3, 4], M[3, 5]],
[M[4, 0], M[4, 1], M[4, 2], M[4, 3], M[4, 4], M[4, 5]],
[M[5, 0], M[5, 1], M[5, 2], M[5, 3], M[5, 4], M[5, 5]]])

In [44]:
Y = DiscreteMarkovChain("Y", ["So1", "Sd1", "Su1", "So2", "Sd2", "Su2"], M)

# symbol Markov chain with symbol matrix for M (thus representing Y)
YS = DiscreteMarkovChain("YS")
trans_YS = TransitionMatrixOf(YS, MS)

In [55]:
# formula: what is the
# probability (P) 
# that being at state 1 at cycle 1 (Eq(YS[1], 1))
# we will be at state 2 at cycle 3 (Eq (YS[3],2))
P(Eq(YS[2], 0),( Eq(YS[0], 0) & trans_YS ) ) 

M[0, 0]**2 + M[0, 1]*M[1, 0] + M[0, 2]*M[2, 0] + M[0, 3]*M[3, 0] + M[0, 4]*M[4, 0] + M[0, 5]*M[5, 0]

In [63]:
# actual execution of the formula (using not YS but Y)
P(Eq(Y[2], 0), (Eq(YS[0], 0)))

0.687500000000000

In [65]:
# Probability of going So1 -> Sd1 -> Su1 -> Su2 -> So1
P(Eq(Y[4], 0) & Eq(Y[3], 5) & Eq(Y[2], 2) & Eq(Y[1], 1), Eq(Y[0], 0))

0.000312500000000000

In [66]:
# Probability of being in So1 or So2 after being in So1
P(Eq(Y[1],0) | Eq(Y[1], 3), Eq(Y[0], 0))

0.950000000000000

In [77]:
P((Eq(Y[2], 0) | Eq(Y[2], 2)) & Eq(Y[1], 2))

Probability(Eq(Y[1], 2) & (Eq(Y[2], 0) | Eq(Y[2], 2)))

In [94]:
t = Eq(Y[2], 0) | Eq(Y[2], 2)
t

Eq(Y[2], 0) | Eq(Y[2], 2)

In [133]:
# state_progression_len = number of individual timesteps (cycles) used in the progression
# if state trace is supposed to be without the starting point - just decrease state_progression_len by 1
def evaluate(prob_func, state_progression_len):
    
    solutions = {
        10: None,
        20: None,
        30: None
    }
    
    s = 0
    # loops 1 - 30 - progression length (inclusive latter val)
    for i in range(1, 30 - (state_progression_len - 1)):
        s += prob_func(i)
        if (i + (state_progression_len)) in solutions.keys():
            solutions[i + (state_progression_len)] = s.round(6)
    return solutions

In [138]:
def print_formatted(solution_dict):
    for (stack_len, probability) in solution_dict.items():
        print('Probability at stack trace length {} is {}'.format(stack_len, probability))

## Questions

In [151]:
# 1 What is the probability of seeing a failure cascade that affects only component 1?

# never, because failure cascade is only possible if multiple components are affected

In [149]:
# 2 What is the probability of seeing a failure cascade that affects both components 1 and 2?

# failure cascades affection component 1 and 2: Sd1 -> Su1 -> Su2, Sd1 -> Sd2 -> Su2

f = lambda i: P((Eq(Y[i+2], 5) & Eq(Y[i+1], 2) & Eq(Y[i], 1)) | (Eq(Y[i+2], 5) & Eq(Y[i+1], 4) & Eq(Y[i], 1)), Eq(Y[0], 0))
print_formatted(evaluate(f, 3))

Probability at stack trace length 10 is 0.014277
Probability at stack trace length 20 is 0.033100
Probability at stack trace length 30 is 0.051962


In [139]:
# 3 What is the probability of seeing failure masking? (P(Su1; Su2; So1))

f = lambda i:  P(Eq(Y[i+2], 0) & Eq(Y[i+1], 5) & Eq(Y[i+0], 2), Eq(Y[0], 0))
print_formatted(evaluate(f, 3))

Probability at stack trace length 10 is 0.002088
Probability at stack trace length 20 is 0.005865
Probability at stack trace length 30 is 0.009638


In [150]:
# 4 What is the probability of seeing a systemic degradation? (P(Sd1 -> Sd2))

f = lambda i: P(Eq(Y[i+1], 4) & Eq(Y[i], 1), Eq(Y[0], 0))
print_formatted(evaluate(f, 2))

Probability at stack trace length 10 is 0.038619
Probability at stack trace length 20 is 0.083963
Probability at stack trace length 30 is 0.129235


In [156]:
# 5 What is the probability of normal operation?

# normal operation = states So1 and So2 are chained together without interruption (allow duplicates)

In [155]:
# probability to stay in So1 or So2 if in So1:
P(Eq(Y[1], 0) | Eq(Y[1], 3), Eq(Y[0], 0))

0.950000000000000

In [157]:
# probability to stay in So1 or So2 if in So2:
P(Eq(Y[1], 0) | Eq(Y[1], 3), Eq(Y[0], 3))

0.950000000000000

In [158]:
# => therefore, the probability to stay in only those two states is 0.95^(lenght of stack trace)
print_formatted({cycles: pow(0.95, cycles) for cycles in [10, 20, 30]})

Probability at stack trace length 10 is 0.5987369392383787
Probability at stack trace length 20 is 0.3584859224085419
Probability at stack trace length 30 is 0.21463876394293727


In [60]:
# 6 What is the probability of having 
#    a) 1, 
#    b) 2, 
#    c) more
# intermittent failures?

# probability of getting exactly one intermittent failure:



In [170]:
# 7 In the case of an intermittent failure, what is the probability of a failure cascade? 
# interpretation is, that the failure cascade follows immediatly on the intermittent failure

# can ignore the intermittent failure in component 2, as this component cannot cause a failure cascade

P(((Eq(Y[5], 5) & Eq(Y[4], 2) & Eq(Y[3], 1)) | (Eq(Y[5], 5) & Eq(Y[4], 4) & Eq(Y[3], 1))) & Eq(Y[2], 0) & Eq(Y[1], 1), Eq(Y[0], 0))

6.15234375000000e-5

In [175]:
# 8 In the case of a failure cascade, what is the probability of failure masking?
# same as above, we assume an immidiatly following failure masking

# there are two possible ways of failure cascade Sd1 -> Sd2 -> Su2 and Sd1 -> Su1 -> Su2
# as failure masking is failing in one component, this cascading into another, 
# and then the other component returning to an operational state in the original component
# only look at second cascading path Sd1 -> Su1 -> Su2

P(Eq(Y[3], 0) & Eq(Y[2], 5) & Eq(Y[1], 2), Eq(Y[0], 1))


0.00625000000000000

In [176]:
# 9 (Extra) In the case of an intermittent failure, what is the probability of failure masking?
# This case means that there is an intermittent failure (in component 1), followed by a failure cascade to Su2,
# followed by failure masking:

# So1 -> Sd1 -> So1 -> Sd1 -> Su1 -> Su2 -> So1

P(Eq(Y[6], 0) & Eq(Y[5], 5) & Eq(Y[4], 2) & Eq(Y[3], 1) & Eq(Y[2], 0) & Eq(Y[1], 1), Eq(Y[0], 0))

7.81250000000000e-6

In [179]:
# this is not completetly equivalent to the product of probabilities from task 7 and 8, as we ignore the case 
# of failure cascading through Sd2, as this would not allow any failure masking to take place 